## <font color=red>Assignment 2</font>

Use this Jupyter Notebook for the first part of the assignment. It contains the model for the Transportation Problem from the exercises. We have studied the version that assumes that the total demand equals the total supply.<BR>

However, in real life, most transportation problems are not balanced. To solved unbalanced problems, one can formulate slightly different models: one for problems with total supply > total demand and one for problems with total supply < total demand. Alternatively, we can just simply adjust the data in case we encounter an unbalanced problem and thereby transforming it in an equivalent balanced problem. More specific, we add either a dummy supply node or a dummy demand node, and use the balanced model to solve it.<BR>

Before diving into the questions for the assignment, do not forget to study the balanced version of the Transportation Problem (see exercise Computer Class 2). The Gurobi model is included here again. See the cell below. In Part (A) use the balanced model to solve an unbalanced problem with data being read from an Excel file. In Part (B) use it for a coaching problem similar to the Linear Assignment Problem. This assignment problem concerns selecting four swimmers for three 4x100m Medley teams (Womens, Mens and Mixed team). See if you select the gold winning team from the European Championship in 2022.

<B>Do not submit this Jupyter Notebook. Only screenshots of the added code are required for the assignment.</B>

In [ ]:
from gurobipy import *

def TransportationModel(supply, demand, cost):

    depots    = range(len(supply))         # Indices for the depots
    customers = range(len(demand))         # Indices for the customers

    if sum(supply) < sum(demand):
        print("Not balanced: supply (", sum(supply), ") < demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    elif sum(supply) > sum(demand):
        print("Not balanced: supply (", sum(supply), ") > demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    
    m = Model("Transportation Problem")
    x = m.addVars(depots, customers, name = "Flow")          # no need to define as INTEGERs (see week 3)
    
    m.setObjective( quicksum(cost[i,j]*x[i,j] for i in depots for j in customers), GRB.MINIMIZE)
    # m.setObjective( x.prod(cost) , GRB.MINIMIZE)               requires dictionaries
    m.addConstrs( (x.sum(i, "*") == supply[i] for i in depots)   , 'Supply')
    m.addConstrs( (x.sum("*", j) == demand[j] for j in customers), 'Demand')
    m.update()
         
    return m

In [ ]:
from gurobipy import *

def TransportationModel(supply, demand, cost):

    depots    = range(len(supply))         # Indices for the depots
    customers = range(len(demand))         # Indices for the customers

    if sum(supply) < sum(demand):
        print("Not balanced: supply (", sum(supply), ") < demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    elif sum(supply) > sum(demand):
        print("Not balanced: supply (", sum(supply), ") > demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    
    m = Model("Transportation Problem")
    x = m.addVars(depots, customers, name = "Flow")          # no need to define as INTEGERs (see week 3)
    
    m.setObjective( quicksum(cost[i,j]*x[i,j] for i in depots for j in customers), GRB.MINIMIZE)
    # m.setObjective( x.prod(cost) , GRB.MINIMIZE)               requires dictionaries
    m.addConstrs( (x.sum(i, "*") == supply[i] for i in depots)   , 'Supply')
    m.addConstrs( (x.sum("*", j) == demand[j] for j in customers), 'Demand')
    m.update()
         
    return m

## <font color=red>(Part A) Use the model with all kinds of data</font>

As in real life, most transportation problems are not balanced, can formulate to similar models: one for problems with total supply > total demand and one for problems with total supply < total demand. Alternatively, we can just simply adjust the data in case we encounter an unbalanced problem. That is, add either a dummy supply node or a dummy demand node.
- (A) do not forget to study the balanced version of the Transportation Problem (see exercise Computer Class 2), 
- (B) use it for an unbalanced problem with data being read from an Excel file,
- (C) and use it for a coaching problem similar to the Linear Assignment Problem. More concrete, four swimmers need to be selected for the three 4x100m Medley teams (Womens, Mens and Mixed team). See if you select the gold winning team from the last European Championship.

### Problem W2.1
In the cell below, data is read from an Excel-file "Transportation.xlsx" for an unbalanced Transportation Problem. Running the cell, you will notice that it will not yield a solution. Modify the code below (not the TransportationModel() function) in order to get an equivalent balanced problem instance and present the solution for the original problem instance.

In [ ]:
import pandas as pd
import numpy as np

m = 5
n = 5
c = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = 1, nrows = m, usecols = 'B:F').values.tolist()
a = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = 1, nrows = m, usecols = 'G').values.flatten().tolist()
b = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = m+1, nrows = 1, usecols = 'B:F').values.tolist()[0]

### Adjust the parameters in order to have a balanced problem that can be solved with TransportationModel()







print("Cost matrix:")
print(c)
print("Supply:", a)
print("Demand:", b)

TP = TransportationModel(a, b, c)
TP.Params.LogToConsole = 0
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()

for var in TP.getVars():
   if abs(var.x) > 1e-6: # only printing non-zeros
       print('{0} = {1:6.2f}'.format(var.VarName[:18], var.x))

## <font color=red>(Part B) Team selection for the 4x100m Medley </font>

### Problem W2.2
In the same Excel file "Transportation.xlsx", the Personal Records of Dutch swimmers who took part in the European Championships of 2022 are provided. Your task is to form teams for the 4x100m relay (Women, Men and Mixed) based on their PR's, i.e. four individuals are selected such that the total PR-time is minimal.

In the cell below, the Women's team is already determined. Adjust the code in order to find the Men's team and the Mixed team. The latter should consist of two male and two female swimmers (check if you have found the winning team).

Note: the problems could be solved by hand rather easily, but that would not be very interesting. Besides that, in reality also reserve-swimmers can be selected for the qualifying heats and the schedule of the championship should be taken in consideration in order to avoid overloading certain swimmers.

In [ ]:
import pandas as pd
from datetime import datetime

m = 13                # Number of Women in TeamNL
n = 5                 # The 5-th event is being a supporter
times = pd.read_excel("Transportation.xlsx", sheet_name = 'Medley', header = None, skiprows = 4, nrows = m, usecols = 'B:E').to_numpy()
    
I = range(m)
J = range(4)
    
a = [1 for i in range(m)]
b = [1 for i in range(n)]
b[n-1] = m - n + 1
c = [[ (60*times[i][j].minute + times[i][j].second + times[i][j].microsecond/1e6) for j in range(n-1)] for i in range(m)] 
[c[i].append(0) for i in range(m)]
c = np.array(c)

print("PB's on the four events")
print(c)
print("Supply:", a, "  Demand", b)

TP = TransportationModel(a, b, c)
# TP.Params.LogToConsole = 0
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()

for j in J:
    for i in I:
        xij = TP.getVarByName(f"Flow[{i},{j}]")
        if xij.x == 1:
            print("Swimmer {:2}   on event {}   PR = {}".format(i+1, j+1, c[i,j]))

# for var in TP.getVars():
#    if abs(var.x) > 1e-6: # only printing non-zeros
#        print('{0} = {1:6.2f}'.format(var.VarName[:18], var.x))